In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import evaluate
from pathlib import Path

NOTEBOOK_DIR = Path("/".join(__vsc_ipynb_file__.split("/")[:-1]))
print(NOTEBOOK_DIR)

/home/kyre/repos/llm-fine-tuning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-19 23:15:53,480] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


/home/kyre/repos/llm-fine-tuning


In [2]:
# https://huggingface.co/datasets/gretelai/synthetic_text_to_sql
dataset = load_dataset("gretelai/synthetic_text_to_sql")

print("Available data subsets:", dataset.keys())
print("Features: ")
for k, v in dataset["train"][0].items():
    print(f"---> {k:30}: {v}")

Available data subsets: dict_keys(['train', 'test'])
Features: 
---> id                            : 5097
---> domain                        : forestry
---> domain_description            : Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.
---> sql_complexity                : single join
---> sql_complexity_description    : only one join (specify inner, outer, cross)
---> sql_task_type                 : analytics and reporting
---> sql_task_type_description     : generating reports, dashboards, and analytical insights
---> sql_prompt                    : What is the total volume of timber sold by each salesperson, sorted by salesperson?
---> sql_context                   : CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_i

In [3]:
train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["train"])

In [4]:
from src.preprocessing import create_sharegpt_format
from pathlib import Path
import json

sharegpt_data = create_sharegpt_format(train_df)

output_filename = Path('data/dataset_train.json')
try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(sharegpt_data, f, indent=2, ensure_ascii=False)
    print(f"Successfully saved ShareGPT formatted data to '{output_filename}'")

    if sharegpt_data:
        print("\n--- First Record Example ---")
        print(json.dumps(sharegpt_data[0], indent=2, ensure_ascii=False))

except Exception as e:
    print(f"Error saving JSON file: {e}")

!mkdir $NOTEBOOK_DIR/LLaMA-Factory/data/
!cp -r $NOTEBOOK_DIR/data/ $NOTEBOOK_DIR/LLaMA-Factory/data/

Successfully saved ShareGPT formatted data to 'data/dataset_train.json'

--- First Record Example ---
{
  "conversations": [
    {
      "from": "user",
      "value": "Context:\n'CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');'\n                                    Question:\n'What is the total volume of timber sold by each salesperson, sorted by salesperson?'"
    },
    {
      "from": "assistant",
      "value": "Result: 'SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name 

In [5]:
max_len_str = 0
for record in sharegpt_data:
    max_len_str = max(max_len_str, len(record['conversations'][0]['value']))
print(max_len_str)

2202


In [6]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd $NOTEBOOK_DIR/LLaMA-Factory
%ls

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.
/home/kyre/repos/llm-fine-tuning/LLaMA-Factory
CITATION.cff    README_zh.md  examples/         src/
LICENSE         assets/       out/              tests/
LLaMA-Factory/  cache/        pyproject.toml    train_qwen3.json
MANIFEST.in     data/         requirements.txt
Makefile        docker/       scripts/
README.md       evaluation/   setup.py


In [7]:
# !GRADIO_SHARE=1 llamafactory-cli webui

In [11]:
training_args = {
    # "deepspeed": DS_CONFIG_PATH,
    "cutoff_len": 1024,
    "dataset": "train_sql_dataset",
    "ddp_timeout": 9000,
    "do_train": True,
    "finetuning_type": "lora",
    "use_dora": True,
    "fp16": True,
    "lora_rank": 16,
    "lora_alpha": 32,
    "flash_attn": "fa2",
    "gradient_accumulation_steps": 8,
    "learning_rate": 5e-5,
    "logging_steps": 8,
    "lora_target": "q_proj,v_proj",
    "lr_scheduler_type": "cosine",
    "model_name_or_path": "Qwen/Qwen3-0.6B",
    "num_train_epochs": 3,
    "output_dir": "out",
    "overwrite_cache": True,
    "overwrite_output_dir": True,
    "per_device_train_batch_size": 4,
    "plot_loss": True,
    "report_to": "wandb", # wnadb
    "save_steps": 100,
    "gradient_checkpointing": True,
    "stage": "sft",
    "template": "qwen3",
    "warmup_steps": 100,
    "weight_decay": 0.1,
    "max_steps": 500
}

json.dump(training_args, open(f"{NOTEBOOK_DIR}/LLaMA-Factory/train_qwen3.json", "w", encoding="utf-8"), indent=2)

In [9]:
!export $(cat $NOTEBOOK_DIR/.env | xargs)
!cd $NOTEBOOK_DIR/LLaMA-Factory

In [ ]:
!llamafactory-cli train LLaMA-Factory/train_qwen3.json


/bin/bash: /home/kyre/repos/llm-fine-tuning/.venv/bin/llamafactory-cli: /home/kyre/repos/llm-fine-tuning/.env/bin/python3: bad interpreter: Not a directory


In [ ]:
!llamafactory-cli export merge_lora.yaml
!llamafactory-cli chat infer_lora.yaml